In [1]:
import pandas as pd 
import numpy as np

In [24]:
# Read CSVs
clicks = pd.read_csv('/home/wb1115/VSCode_projects/cdss/cdss_1/cdss_1/click_results/demo_clicks.csv')
hover_events = pd.read_csv('/home/wb1115/VSCode_projects/cdss/cdss_1/cdss_1/hover_results/demo_hover_events.csv')
page_visits = pd.read_csv('/home/wb1115/VSCode_projects/cdss/cdss_1/cdss_1/timing_results/demo_page_visits.csv')

hover_events

In [4]:
hover_events

,user_id,user_archetype,patient,component,hover_duration
0,demo,a,1,patient_details_age:,0.033
1,demo,a,1,patient_details_sex:,0.016
2,demo,a,1,patient_details_ethnicity:,0.047
3,demo,a,1,patient_details_imddecil:,0.133
4,demo,a,1,patient_details_co-morbidities:,0.067
...,...,...,...,...,...
205,demo,a,3,vitals_table_SpO2,0.088
206,demo,a,3,vitals_table_SpO2,0.016
207,demo,a,3,vitals_table_Mean Blood Pressure,0.037
208,demo,a,3,vitals_table_Mean Blood Pressure,0.072


In [9]:
component_hover_duration = hover_events.groupby('component')['hover_duration'].sum().reset_index()
component_hover_duration = component_hover_duration.sort_values(by='hover_duration', ascending=False).reset_index(drop=True)
component_hover_duration

,component,hover_duration
0,echarts_plot,13.027
1,feature_image,5.265
2,guidelines_flowchart,4.840
3,guidelines_pdf,3.047
4,vitals_table_NEWS Conscious Level Score,2.130
5,similar_table_3,2.008
6,vitals_table_SpO2,1.865
7,similar_table_2,1.801
8,patient_table,1.416
9,vitals_table_Glasgow Coma Score - verbal response,1.325


In [11]:
# Grouping components into broader catergories 

# Define a function to categorize the components
def categorize_component(component_str):
    categories = ['vitals_table', 'similar_table', 'patient_details', 'guidelines_pdf', 'LLM']
    for category in categories:
        if category in component_str:
            return category
    return component_str

# Apply the categorization function to create the new column
hover_events['component_categories'] = hover_events['component'].apply(categorize_component)
component_hover_duration = hover_events.groupby('component_categories')['hover_duration'].sum().reset_index()
component_hover_duration = component_hover_duration.sort_values(by='hover_duration', ascending=False).reset_index(drop=True)
component_hover_duration

,component_categories,hover_duration
0,echarts_plot,13.027
1,vitals_table,8.651
2,similar_table,6.710
3,feature_image,5.265
4,guidelines_flowchart,4.840
5,guidelines_pdf,3.746
6,patient_details,1.733
7,patient_table,1.416
8,prediction_alert,1.192
9,LLM,0.957


In [13]:
# Grouping by patient as well

component_hover_duration = hover_events.groupby(['component_categories', 'patient'])['hover_duration'].sum().reset_index()
component_hover_duration = component_hover_duration.sort_values(by='hover_duration', ascending=False).reset_index(drop=True)
component_hover_duration

,component_categories,patient,hover_duration
0,echarts_plot,1,11.666
1,vitals_table,1,4.788
2,similar_table,1,4.231
3,vitals_table,3,3.254
4,feature_image,3,3.081
5,similar_table,3,2.479
6,guidelines_flowchart,1,2.431
7,guidelines_flowchart,3,2.409
8,feature_image,1,2.184
9,guidelines_pdf,1,1.998


In [ ]:
# In future can stack different df from different patricipants, group and then analyze 

page_visits

In [27]:
# Convert 'entry_time' and 'exit_time' columns to datetime
page_visits['entry_time'] = pd.to_datetime(page_visits['entry_time'])
page_visits['exit_time'] = pd.to_datetime(page_visits['exit_time'])

# Iterate over DataFrame and update exit_time
for index, row in page_visits.iterrows():
    if row['page_name'] == '/record_hover_event/' and index > 0:
        previous_exit_time = page_visits.at[index - 1, 'exit_time']
        if row['exit_time'] > previous_exit_time:
            page_visits.at[index - 1, 'exit_time'] = row['exit_time']

# Drop pages / rows we dont care about
page_visits = page_visits[~page_visits['page_name'].str.contains('favicon|process_user_input|track_link_click|record_hover_event')]
page_visits = page_visits[page_visits['page_name'] != '/']

# Calculate visit duration in seconds
page_visits['visit_duration'] = (page_visits['exit_time'] - page_visits['entry_time']).dt.total_seconds()

# Group by
grouped_page_visits = page_visits.groupby(['page_name'])['visit_duration'].sum().reset_index()
grouped_page_visits = grouped_page_visits.sort_values(by='visit_duration', ascending=False).reset_index(drop=True)
grouped_page_visits



,page_name,visit_duration
0,/1_demo/,40.161503
1,/3_demo/,24.704385
2,/1_demo/prediction/,15.631954
3,/3_demo/prediction/,14.009150
4,/patients_demo,9.567636
5,/2_demo/,8.027680
6,/1_demo/guideline/,6.923755
7,/3_demo/guideline/,5.403025


In [36]:
# Grouping page_name into patients

# Define a function to group page_name by patients
def categorize_page_name(component_str):
    categories = [str(element) for element in list(range(13))]
    for category in categories:
        if category in component_str:
            return category
    return component_str

# Apply the categorization function to create the new column
page_visits['page_name_categories'] = page_visits['page_name'].apply(categorize_page_name)
grouped_page_visits = page_visits.groupby('page_name_categories')['visit_duration'].sum().reset_index()
grouped_page_visits = grouped_page_visits.sort_values(by='visit_duration', ascending=False).reset_index(drop=True)
grouped_page_visits

,page_name_categories,visit_duration
0,1,62.717212
1,3,44.116560
2,/patients_demo,9.567636
3,2,8.027680


In [37]:
page_visits

,user_id,user_archetype,page_name,entry_time,exit_time,visit_duration,page_name_categories
1,demo,a,/patients_demo,2024-03-13 15:52:43.810713+00:00,2024-03-13 15:52:43.852561+00:00,0.041848,/patients_demo
3,demo,a,/1_demo/,2024-03-13 15:52:45.672470+00:00,2024-03-13 15:53:07.986090+00:00,22.313620,1
6,demo,a,/1_demo/prediction/,2024-03-13 15:53:08.014844+00:00,2024-03-13 15:53:23.646798+00:00,15.631954,1
8,demo,a,/1_demo/,2024-03-13 15:53:23.664754+00:00,2024-03-13 15:53:30.087907+00:00,6.423153,1
11,demo,a,/1_demo/guideline/,2024-03-13 15:53:30.108387+00:00,2024-03-13 15:53:37.032142+00:00,6.923755,1
15,demo,a,/1_demo/,2024-03-13 15:53:45.065539+00:00,2024-03-13 15:53:56.490269+00:00,11.424730,1
18,demo,a,/patients_demo,2024-03-13 15:53:56.514283+00:00,2024-03-13 15:53:58.959039+00:00,2.444756,/patients_demo
19,demo,a,/2_demo/,2024-03-13 15:53:58.969356+00:00,2024-03-13 15:54:06.997036+00:00,8.027680,2
24,demo,a,/patients_demo,2024-03-13 15:54:16.099222+00:00,2024-03-13 15:54:20.218725+00:00,4.119503,/patients_demo
25,demo,a,/3_demo/,2024-03-13 15:54:20.224338+00:00,2024-03-13 15:54:34.378743+00:00,14.154405,3


clicks

In [38]:
clicks

,user_id,user_archetype,patient,link_clicks,timestamp
0,demo,a,1,predictiondetailsLink,2024-03-13 15:53:07.976616+00:00
1,demo,a,1,guidelinedetailsLink,2024-03-13 15:53:30.080327+00:00
2,demo,a,1,guidelinesLink,2024-03-13 15:53:37.017701+00:00
3,demo,a,2,guidelinesLink_simple,2024-03-13 15:54:06.985623+00:00
4,demo,a,3,predictiondetailsLink,2024-03-13 15:54:34.368593+00:00
5,demo,a,3,guidelinedetailsLink,2024-03-13 15:54:51.712688+00:00
6,demo,a,3,guidelinesLink,2024-03-13 15:54:57.139433+00:00


In [40]:
clicks['link_clicks'].value_counts()

link_clicks
predictiondetailsLink    2
guidelinedetailsLink     2
guidelinesLink           2
guidelinesLink_simple    1
Name: count, dtype: int64

In [41]:
clicks.groupby('patient')['link_clicks'].value_counts()

patient  link_clicks          
1        guidelinedetailsLink     1
         guidelinesLink           1
         predictiondetailsLink    1
2        guidelinesLink_simple    1
3        guidelinedetailsLink     1
         guidelinesLink           1
         predictiondetailsLink    1
Name: count, dtype: int64

In [42]:
# Define a function to group page_name by patients
def categorize_link_clicks(component_str):
    categories = ['guidelinesLink']
    for category in categories:
        if category in component_str:
            return category
    return component_str

# Apply the categorization function to create the new column
clicks['link_clicks_categories'] = clicks['link_clicks'].apply(categorize_link_clicks)
clicks['link_clicks_categories'].value_counts()

link_clicks_categories
guidelinesLink           3
predictiondetailsLink    2
guidelinedetailsLink     2
Name: count, dtype: int64

In [ ]:
# In future add column to df based on user_archetype and patient 
# to determine if they saw 'soc' or 'cdss'
# can then groubby that for interesting results! 